In [48]:
# Import necessary packages and libraries
import pandas as pd
import json
from PIL import Image, ImageDraw, ImageFont
from PIL.ImageOps import scale
import math
from itertools import groupby
import numpy as np
from statistics import mean

In [49]:
# Initialize the sample dataset and experimental labels to respective dataframes
df = pd.read_excel('Modified Tissue Archive for Graphical Abstract Generation.xlsx')
df2 =  pd.read_excel('Experiment Info.xlsx')

In [50]:
# Redefine the first row to be the row labels for each data point
group_data = df[1:]
group_data.columns = df.iloc[0]
group_data

,Expt group,group-subgroup,Mouse ID,Campaign,Cage,Rad type,Dose (Gy),Sex,Countermeasure,DOB,IrradDate,Date of death,Recovery (d)
1,H,H1,643,19A,187,γ-ray,0.75,M,N,2018-11-06 00:00:00,2019-04-23 00:00:00,2019-05-21 00:00:00,28
2,H,H1,644,19A,187,γ-ray,0.75,M,N,2018-11-06 00:00:00,2019-04-23 00:00:00,2019-05-21 00:00:00,28
3,H,H1,645,19A,187,γ-ray,0.75,M,N,2018-11-06 00:00:00,2019-04-23 00:00:00,2019-05-21 00:00:00,28
4,H,H1,651,19A,231,γ-ray,0.75,M,N,2018-11-06 00:00:00,2019-04-23 00:00:00,2019-05-21 00:00:00,28
5,H,H2,693,19A,239,γ-ray,0.75,F,N,2018-11-06 00:00:00,2019-04-23 00:00:00,2019-05-21 00:00:00,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...
610,I,I6,779,19A,199,γ-ray,1.5,M,No,2018-11-06 00:00:00,2019-04-23 00:00:00,2020-06-12 00:00:00,416
611,I,I6,780,19A,199,γ-ray,1.5,M,No,2018-11-06 00:00:00,2019-04-23 00:00:00,2020-08-06 00:00:00,471
612,I,I5,696,18C,39A,γ-ray,1.5,M,No,2018-05-22 00:00:00,2018-11-05 00:00:00,2019-10-04 00:00:00,333
613,I,I5,697,18C,39,γ-ray,1.5,M,No,2018-05-22 00:00:00,2018-11-05 00:00:00,2019-10-04 00:00:00,333


In [51]:
df2

,Pi Name,PI Institution,Experiment Title,Location of Experiment,Experimental Type,Species,Strain,Genotype
0,Bill Dynan,Emory University,The Effect of GCR Sim on Tumorigenesis,Brookhaven National Labs (NSRL),Radiation,Mus Musculus,C57BL/6J,Wild Type


In [52]:
class BackgroundTemplate:
    '''
    Description: Background Template helps to define box methods to create the graphical abstract template 
    which will house all the necessary information needed for the graphical abstract visualizer
    '''
    def __init__(self, image_size, background_color='white'):
        self.image = Image.new('RGB', image_size, color=background_color)
        self.draw = ImageDraw.Draw(self.image)
        self.boxes = []

    def add_box(self, top_left, bottom_right, outline_color='black', fill_color=None, text=None, text_color='black'):
        box = {'top_left': top_left, 'bottom_right': bottom_right, 'outline_color': outline_color, 'fill_color': fill_color, 'text': text, 'text_color': text_color}
        self.boxes.append(box)

    def draw_boxes(self):
        for box in self.boxes:
            self.draw.rectangle([box['top_left'], box['bottom_right']], outline=box['outline_color'], fill=box['fill_color'])
            if box['text']:
                self._draw_centered_text(box['text'], box['top_left'], box['bottom_right'], box['text_color'])

    def _draw_centered_text(self, text, top_left, bottom_right, text_color, font = None):
        if font == None:
            font_path = "./fonts/ARIAL.TTF"
            font_size = 13
            font = ImageFont.truetype(font_path, font_size)
        bbox = self.draw.textbbox((0, 0), text, font=font)
        text_width = bbox[2] - bbox[0]
        text_height = bbox[3] - bbox[1]
        box_width = bottom_right[0] - top_left[0]
        box_height = bottom_right[1] - top_left[1]
        x = top_left[0] + (box_width - text_width) / 2
        y = top_left[1] + (box_height - text_height) / 2
        self.draw.text((x, y), text, fill=text_color, font=font)


    def get_image(self):
        return self.image

    def add_enclosed_boxes(self, enclosing_box_top_left, enclosing_box_bottom_right, num_inner_boxes, spacing, colors, texts):
        # Define the enclosing box
        self.add_box(enclosing_box_top_left, enclosing_box_bottom_right, outline_color='black', fill_color=None)
        
        # Calculate inner box dimensions and spacing
        enclosing_width = enclosing_box_bottom_right[0] - enclosing_box_top_left[0]
        enclosing_height = enclosing_box_bottom_right[1] - enclosing_box_top_left[1]
        box_width = enclosing_width - 2 * spacing
        box_height = (enclosing_height - (num_inner_boxes + 1) * spacing) // num_inner_boxes
        
        # Add inner boxes
        for i in range(num_inner_boxes):
            top_left = (enclosing_box_top_left[0] + spacing, enclosing_box_top_left[1] + spacing + i * (box_height + spacing))
            bottom_right = (top_left[0] + box_width, top_left[1] + box_height)
            self.add_box(top_left, bottom_right, outline_color='black', fill_color=colors[i % len(colors)], text=texts[i], text_color='white')


In [53]:
def add_labels(template, group_data, initial_pos, initial_age, pos_y):

    '''
    Description: Labels the age of mice during the treatment procedure from initial age to 
    final termination. This function dynamically scales the number of weeks in or out based on the 
    largest termination date present in the dataset. 
    '''
    
    font_path = "./fonts/ARIAL.TTF"
    font_size = 10
    font = ImageFont.truetype(font_path, font_size)
    
    weeks_to_display = []       
    weeks_to_display += [int(math.ceil(float(group) / 7.0)) for group in group_data['Recovery (d)']]
    max_duration = max(weeks_to_display)

    pos_x = initial_pos
    prev = 0

    if max_duration <= 20: 
        week_labels = range(initial_age, (initial_age+max_duration), 1)
    elif max_duration <= 50:
        week_labels = range(initial_age, (initial_age+max_duration), 2) 
    elif max_duration <= 200:
        week_labels = range(initial_age, (initial_age+max_duration), 5)
    else:
        week_labels = range(initial_age, (initial_age+max_duration), 10)

    template._draw_centered_text("Week", (pos_x - 70, pos_y), (pos_x + 20, pos_y), text_color = 'black', font = font)
    
    for week in week_labels:
        pos_x += ((1300 - initial_pos) / max_duration) * (week - prev - initial_age)
        template._draw_centered_text(f"{week}", (pos_x, pos_y), (pos_x + 20, pos_y), text_color = 'black', font = font)
        prev = week - initial_age
    
    pos_x += ((1300 - initial_pos) / max_duration) * (max_duration - prev)
    template._draw_centered_text(f"{max_duration+initial_age}", (pos_x, pos_y), (pos_x + 20, pos_y), text_color = 'black', font = font)
    prev = week - initial_age
    
        

In [54]:
# # Make this a dynamic position
# def add_labels(template, group_data, initial_pos, pos_y):
    
#     weeks_to_display = []       
#     weeks_to_display += [int(math.ceil(float(group) / 7.0)) for group in group_data['Recovery (d)']]
#     new_weeks_to_display = np.unique(weeks_to_display)

#     max_duration = max(weeks_to_display)
    
#     pos_x = initial_pos
#     prev = 0

#     font_path = "./fonts/ARIAL.TTF"
#     font_size = 6
#     font = ImageFont.truetype(font_path, font_size)
    
#     for week in new_weeks_to_display:
#         print(week)
#         pos_x += ((1300 - initial_pos) / max_duration) * (week - prev)
#         week_new = 24 + week
#         template._draw_centered_text(f"Wk {week_new}", (pos_x, pos_y), (pos_x + 20, pos_y), text_color = 'black', font = font)
#         prev = week

#     return "Labeling Complete"

In [55]:
def duration_to_arrow_length(template, group_data, x, y, y_increment, min_length=40, max_length=670):
            
    
    
    recovery_times = group_data.groupby(['Rad type','Expt group', 'group-subgroup', 'Dose', 'Sex', 'Countermeasure'])['Recovery (d)'].mean().reset_index()
    recovery_times = recovery_times.sort_values(by='Sex')
    print(recovery_times)
    
    min_weeks = 1
    max_weeks = math.ceil(float(max(recovery_times['Recovery (d)'])) / 7.0)
    for days in recovery_times['Recovery (d)']:
        week = math.ceil(float(days) / 7.0)
        length = ((week - min_weeks) / (max_weeks - min_weeks)) * (max_length - min_length) + min_length
        arrow_length = max(min_length, min(length, max_length))
        arrow_start = (x , y)
        arrow_end = (arrow_start[0] + arrow_length, arrow_start[1])
        template.draw.line([arrow_start, arrow_end], fill='black', width=1)
        arrowhead = [(arrow_end[0], arrow_end[1]), (arrow_end[0] - 2, arrow_end[1] - 2),
                (arrow_end[0] - 2, arrow_end[1] + 2)]
        template.draw.polygon(arrowhead, fill='black')
        y += y_increment
         
    return "Drawing Arrows Complete"

In [56]:
 def draw_arrow(template, recovery_time, x, y, min_length=40, max_length=670):
    recovery_times = group_data.groupby(['Rad type','Expt group', 'group-subgroup', 'Dose (Gy)', 'Sex', 'Countermeasure'])['Recovery (d)'].mean().reset_index()
    recovery_times = recovery_times.sort_values(by=['Expt group','Countermeasure', 'Sex'])
     
    min_weeks = 1
    max_weeks = math.ceil(float(max(recovery_times['Recovery (d)'])) / 7.0)
    week = math.ceil(float(recovery_time) / 7.0)
    length = ((week - min_weeks) / (max_weeks - min_weeks)) * (max_length - min_length) + min_length
    arrow_length = max(min_length, min(length, max_length))
    arrow_start = (x , y)
    arrow_end = (arrow_start[0] + arrow_length, arrow_start[1])
    template.draw.line([arrow_start, arrow_end], fill='black', width=1)
    arrowhead = [(arrow_end[0], arrow_end[1]), (arrow_end[0] - 2, arrow_end[1] - 2),
            (arrow_end[0] - 2, arrow_end[1] + 2)]
    template.draw.polygon(arrowhead, fill='black')
         
    return "Arrow Drawn"

In [57]:
def create_experiment_structure(template):

    # Create the dataframes for labelling the template
    df = pd.read_excel('Modified Tissue Archive for Graphical Abstract Generation.xlsx')
    df2 =  pd.read_excel('Experiment Info.xlsx')

    # Modify the dataframes to have the right headers
    group_data = df[1:]
    group_data.columns = df.iloc[0]

    # Sort and Group the Dataframe according to the Radiation, Experimental Group / Subgroup, Gender and Countermeasure
    recovery_times = group_data.groupby(['Rad type','Expt group', 'group-subgroup', 'Dose (Gy)', 'Sex', 'Countermeasure'])['Recovery (d)'].mean().reset_index()
    recovery_times = recovery_times.sort_values(by=['Expt group','Countermeasure', 'Sex'])
    recovery_times

    rad_types = list(recovery_times['Rad type'].unique())
    experimental_groups = list(recovery_times['Dose (Gy)'].unique())
    counter_groups = list(recovery_times['Countermeasure'].unique())
    gender_groups = list(recovery_times['Sex'].unique())

    x = 630
    y = 100
    arrow_y = []
    gender_y = []
    counter_y = []
    group_y = []
    rad_y = []

    for rad_type in rad_types:
        for group in experimental_groups:
            filtered_df = recovery_times.loc[recovery_times['Dose (Gy)'] == group]
            if filtered_df.empty:
                continue
            else: 
                print(filtered_df)
                for counter in counter_groups:
                    counter_df = filtered_df.loc[filtered_df['Countermeasure'] == counter]
                    if counter_df.empty:
                        continue
                    else: 
                        print(counter_df)
                        for gender in gender_groups: 
                            gender_df = counter_df.loc[counter_df['Sex'] == gender]
                            if gender_df.empty:
                                continue
                            else: 
                                print(gender_df)
                                for index, row in gender_df.iterrows():
                                    draw_arrow(template, row['Recovery (d)'], x, y, min_length=40, max_length=670)
                                    arrow_y.append(y)
                                    y += 10
                            min_g = min(arrow_y)
                            max_g = max(arrow_y)
                            if gender == 'M':
                                gender = 'Male'
                            else:
                                gender = 'Female'
                            template._draw_centered_text(f"Gender: {gender}", (550, min_g), (550 + 20, max_g), text_color = 'black')
                            counter_y.append(mean([min_g, max_g]))
                            arrow_y = []
                            y += 20
                    min_c = min(counter_y)
                    max_c = max(counter_y)
                    template._draw_centered_text(f"Countermeasure: {counter}", (500, min_c), (500 + 20, max_c), text_color = 'black')
                    group_y.append(mean([min_c, max_c]))
                    counter_y = []
                    y += 40
                    print(y)
            min_gr = min(group_y)
            max_gr = max(group_y)

            print("This is the min and max group: ", min_gr, " | ", max_gr)
            
            template._draw_centered_text(f"Dosage: {group}", (450, min_gr), (450 + 20, max_gr), text_color = 'black')
            rad_y.append(mean([min_gr, max_gr]))
            group_y = []
            y += 80
        min_r = min(rad_y)
        max_r = max(rad_y)
        template._draw_centered_text(f"{rad_type}", (450, min_r), (450 + 20, max_r), text_color = 'black')
        y += 160

In [58]:
def generate_image(pi_info, exp_info, group_data):
    
        # Experiment Initialization
        template = BackgroundTemplate((1400, 750))
        
        font_path = "./fonts/ARIAL.TTF"
        font_size = 8
        font = ImageFont.truetype(font_path, font_size)

        # Define the parameters for the first set of enclosed boxes
        enclosing_box_top_left1 = (50, 50)
        enclosing_box_bottom_right1 = (350, 300)
        num_inner_boxes = 4
        spacing = 20
        colors = ['grey', 'grey', 'grey', 'grey']
        texts1 = pi_info

        # Add the first set of enclosed boxes
        template.add_enclosed_boxes(enclosing_box_top_left1, enclosing_box_bottom_right1, num_inner_boxes, spacing, colors, texts1)

        # Define the parameters for the second set of enclosed boxes
        enclosing_box_top_left2 = (50, 320)
        enclosing_box_bottom_right2 = (350, 570)
        texts2 = exp_info

        # Add the second set of enclosed boxes
        template.add_enclosed_boxes(enclosing_box_top_left2, enclosing_box_bottom_right2, num_inner_boxes, spacing, colors, texts2)

        # Define the parameters for the Treatment Group Visualizer
        treatment_box_top_left = (370, 50)
        treatment_box_bottom_right = (1350, 650)
        template.add_box(treatment_box_top_left, treatment_box_bottom_right)
        
        #Display Weeks
        add_labels(template, group_data, 630, 24, 70)
    
        # duration_to_arrow_length(template, group_data, 630, 120, 10)
        create_experiment_structure(template)
    
        # Draw all boxes
        template.draw_boxes()
        # Display or save the image
        template.get_image().show()  # Display the image
        # template.get_image().save('background_template.png')  # Save the image\


In [59]:
total_info = df2.iloc[0]
pi_info = total_info[:4]
exp_info = total_info[4:]

In [61]:
generate_image(pi_info, exp_info, group_data)

/var/folders/8z/xh7nv8fd54lb36dkhtrtmvsh0000gp/T/ipykernel_14064/2869967204.py:53: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  self.add_box(top_left, bottom_right, outline_color='black', fill_color=colors[i % len(colors)], text=texts[i], text_color='white')


  Rad type Expt group group-subgroup  Dose (Gy) Sex Countermeasure  \
1  GCR sim          E             E2       0.75   F              N   
3  GCR sim          E             E4       0.75   F              N   
5  GCR sim          E             E6       0.75   F              N   
0  GCR sim          E             E1       0.75   M              N   
2  GCR sim          E             E3       0.75   M              N   
4  GCR sim          E             E5       0.75   M              N   

  Recovery (d)  
1         31.0  
3        185.0  
5   506.255682  
0         30.0  
2        184.0  
4   536.943182  
  Rad type Expt group group-subgroup  Dose (Gy) Sex Countermeasure  \
1  GCR sim          E             E2       0.75   F              N   
3  GCR sim          E             E4       0.75   F              N   
5  GCR sim          E             E6       0.75   F              N   
0  GCR sim          E             E1       0.75   M              N   
2  GCR sim          E             E3    